In [30]:
####################################################################################
# This test code shows how new_basin_cities looks like
# basin at each city
distance=100
min_of_cities=0
max_of_basinnum=1000
####################################################################################
h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
basin_cities_path = f'{h08dir}/global_city/dat/riv_num_/basin_to_cities_{distance}km.json'

with open(basin_cities_path, 'r') as json_file:
    tmp = json.load(json_file)
    
sorted_keys = sorted(tmp.keys(), key=lambda k: float(k))

target_city = 77
target_key_list = []

for key in sorted_keys:
    value = tmp[key]
    if min_of_cities < len(value) and float(key) < max_of_basinnum:
        for i in value:
            if int(float(i)) == target_city:
                target_key_list.append(key)
        #print(f'key{key}, len: {len(value)}')
        #print(value)
        #print('-----------------------------------------')
print(f'target_city: {target_city}')
print(f'target_key_list: {target_key_list}')

target_city: 77
target_key_list: ['21.0', '43.0']


In [35]:
target_key = '21.0'
value = tmp[target_key]
print(f'target_key: {target_key}')
print(f'value: {value}')

target_key: 21.0
value: ['77', '122', '256', '656', '1060', '1513', '1625']
